<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv1D,Conv2D, BatchNormalization, Dropout, Activation, Softmax, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, RMSprop

import numpy as np
import pandas as pd
import os
import random
import pprint

from  matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

Mounted at /content/drive


In [ ]:
!pip install wandb
!wandb login
import wandb
from wandb.keras import WandbCallback
wandb.init(project="CS6910-assg2", entity="swathi")

     |████████████████████████████████| 2.1MB 6.0MB/s 
     |████████████████████████████████| 163kB 22.4MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 133kB 19.7MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=138179ffe39b080b98f3f41b9fe8e4d32de9220b614b210045ef4f6813ec466d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=01d09b2ea624b221f5a35166b6cf76e5b0563b74ff758871d6e943eb2a17e115
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 

In [6]:
def load_data(dir_train, dir_test, batch):
  
  seed = 42
  
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    horizontal_flip = True,
    rotation_range = 30,
    validation_split = 0.1)
  
  val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    validation_split = 0.1)
  
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0)
  
  train_aug_dataset = train_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)

  train_dataset = val_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)
  
  val_dataset = val_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'validation',
    seed = seed)
  
  test_dataset = test_datagen.flow_from_directory(
    dir_test,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = None,
    seed = seed
  )
  
  return train_aug_dataset, train_dataset, val_dataset, test_dataset

In [5]:
def CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):

  # Input layer
  X_input = keras.Input(shape=(img_dim,img_dim,channel_no))

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [7]:
def CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):

  # Input layer
  X_input = keras.Input(shape=(img_dim,img_dim,channel_no))

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [8]:
def model_choose(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, batch_norm, learning_rate):
  wandb.log({"Filters": filter_matrix})
  if batch_norm == True:
    model = CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  else:
    model = CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  
  return model

In [ ]:
@tf.custom_gradient
def GBRelu(x):
  def gradient(dy):
    grad = tf.cast(dy>0,"float32")*tf.cast(x>0, "float32")*dy
    return  grad
  return tf.nn.relu(x), gradient
# Here model is the best model
GBModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=8).output]) # Here the index is to be checked
layer_dict = [layer for layer in gb_model.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
  if layer.activation == tf.keras.activations.relu:
    layer.activation = GBRelu

with tf.GradientTape() as tape:
  inputs = tf.cast(X_input, tf.float32)
  tape.watch(inputs)
  outputs = GBModel(inputs)

grads = tape.gradient(outputs,inputs)[0]

In [9]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'filter_matrix':{
        'values': [2]
      },
    'kernel_matrix': {
        'values': [0]
      },
    'activation_matrix': {
        'values': [0]
      },
    'learn_rate': {
        'values': [1e-3]
      },
    'epochs': {
        'values': [80]
      },
    'dropout': {
        'values': [0.2]
      },
    'batch_normalization': {
        'values': [True]
      },
    'dense_number': {
          'values': [512]
      },
    'augmentation':{
        'values' : [True]
      }
}
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def training_sweep(config=None):
    with wandb.init(config=config):
        config = wandb.config

        F = filters[config.filter_matrix]
        K = kernels[config.kernel_matrix]
        A = activations[config.activation_matrix]

        model = model_choose(F, K, A, config.dropout, config.dense_number, config.batch_normalization, config.learn_rate)
        if config.augmentation == True:
          train = train_aug_dataset
        else:
          train = train_dataset

        history = model.fit(train, 
                            epochs=config.epochs,
                            validation_data = val_dataset,
                            callbacks = [WandbCallback(data_type='image', labels = classes)]
                            )
        wandb.log(history.history)

{'method': 'grid',
 'parameters': {'activation_matrix': {'values': [0]},
                'augmentation': {'values': [True]},
                'batch_normalization': {'values': [True]},
                'dense_number': {'values': [512]},
                'dropout': {'values': [0.2]},
                'epochs': {'values': [80]},
                'filter_matrix': {'values': [2]},
                'kernel_matrix': {'values': [0]},
                'learn_rate': {'values': [0.001]}}}


In [10]:
global img_dim
img_dim = 200

global channel_no
channel_no = 3 #3 for RGB images, 1 for greyscale

global batch
batch = 128

global filters
filters = [[64,128,256,512,512],[64,128,256,256,512],[128,256,256,512,512],[96,128,256,512,512]]

global kernels
kernels = [[(3,3)]*5]

global activations
activations = [['relu']*6]

global F, K, A

global train_aug_dataset, train_dataset, val_dataset, test_datagen
train_aug_dataset, train_dataset, val_dataset, test_dataset = load_data('/content/drive/MyDrive/inaturalist_12K/train', '/content/drive/MyDrive/inaturalist_12K/val', batch)

Found 9018 images belonging to 10 classes.
Found 9018 images belonging to 10 classes.
Found 1001 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg2")
wandb.agent(sweep_id, training_sweep)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ci6pf4cp
Sweep URL: https://wandb.ai/swathi/CS6910-assg2/sweeps/ci6pf4cp


wandb: Agent Starting Run: zbc1khhk with config:
wandb: 	activation_matrix: 0
wandb: 	augmentation: True
wandb: 	batch_normalization: True
wandb: 	dense_number: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 80
wandb: 	filter_matrix: 2
wandb: 	kernel_matrix: 0
wandb: 	learn_rate: 0.001


Epoch 1/80
71/71 [==============================] - 3651s 51s/step - loss: 5.9686 - accuracy: 0.1287 - val_loss: 2.2951 - val_accuracy: 0.1209
Epoch 2/80
71/71 [==============================] - 239s 3s/step - loss: 2.2129 - accuracy: 0.1777 - val_loss: 2.3100 - val_accuracy: 0.1109
Epoch 3/80
71/71 [==============================] - 222s 3s/step - loss: 2.1515 - accuracy: 0.2180 - val_loss: 2.3578 - val_accuracy: 0.1099
Epoch 4/80
71/71 [==============================] - 216s 3s/step - loss: 2.0885 - accuracy: 0.2554 - val_loss: 2.3771 - val_accuracy: 0.1489
Epoch 5/80
71/71 [==============================] - 217s 3s/step - loss: 2.0522 - accuracy: 0.2495 - val_loss: 2.3987 - val_accuracy: 0.1369
Epoch 6/80
71/71 [==============================] - 221s 3s/step - loss: 2.0175 - accuracy: 0.2737 - val_loss: 2.3232 - val_accuracy: 0.1558
Epoch 7/80
71/71 [==============================] - 219s 3s/step - loss: 1.9970 - accuracy: 0.2866 - val_loss: 2.1310 - val_accuracy: 0.2058
Epoch 8/80


In [1]:
wandb.agent(sweep_id, training_sweep)

NameError: ignored